# Assignment 4: Quantum Channel Classification

## Task 1 · Environment check

In [20]:
import numpy as np
import pandas as pd
import joblib
import sklearn
import qiskit

print("numpy:", np.__version__)
print("pandas:", pd.__version__)
print("sklearn:", sklearn.__version__)
print("qiskit:", qiskit.__version__)
print("joblib:", joblib.__version__)

numpy: 2.4.0
pandas: 2.3.3
sklearn: 1.8.0
qiskit: 2.2.3
joblib: 1.5.3


## Task 2 · Import helper modules

In [21]:
import numpy as np
import pandas as pd
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from qiskit.quantum_info import Kraus, Choi

## Task 3 · Build a calibration-time classifier

In [22]:
def build_channel_classifier():
    """
    Trains a lightweight classifier to distinguish
    depolarizing vs amplitude damping channels.
    """

    rng = np.random.default_rng(0)

    # --- Pauli matrices ---
    I = np.eye(2, dtype=complex)
    X = np.array([[0, 1], [1, 0]], dtype=complex)
    Y = np.array([[0, -1j], [1j, 0]], dtype=complex)
    Z = np.array([[1, 0], [0, -1]], dtype=complex)

    # --- Channel definitions ---
    def depolarizing_kraus(p):
        k0 = np.sqrt(1 - p) * I
        k1 = np.sqrt(p / 3) * X
        k2 = np.sqrt(p / 3) * Y
        k3 = np.sqrt(p / 3) * Z
        return Kraus([k0, k1, k2, k3])

    def amplitude_damping_kraus(gamma):
        k0 = np.array([[1, 0], [0, np.sqrt(1 - gamma)]], dtype=complex)
        k1 = np.array([[0, np.sqrt(gamma)], [0, 0]], dtype=complex)
        return Kraus([k0, k1])

    def channel_to_feature(channel):
        choi = Choi(channel).data
        return np.concatenate([choi.real.flatten(), choi.imag.flatten()])

    # --- Dataset generation ---
    X_data = []
    y_data = []

    for p in np.linspace(0.05, 0.8, 40):
        X_data.append(channel_to_feature(depolarizing_kraus(p)))
        y_data.append("depolarizing")

    for g in np.linspace(0.05, 0.8, 40):
        X_data.append(channel_to_feature(amplitude_damping_kraus(g)))
        y_data.append("amplitude_damping")

    X_data = np.array(X_data)
    y_data = np.array(y_data)

    # --- Train / validation split ---
    X_train, X_val, y_train, y_val = train_test_split(
        X_data, y_data, test_size=0.25, random_state=0, stratify=y_data
    )

    # --- Model ---
    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)

    # --- Report ---
    preds = model.predict(X_val)
    print("Validation report:")
    print(classification_report(y_val, preds))

    return model

model = build_channel_classifier()

Validation report:
                   precision    recall  f1-score   support

amplitude_damping       0.91      1.00      0.95        10
     depolarizing       1.00      0.90      0.95        10

         accuracy                           0.95        20
        macro avg       0.95      0.95      0.95        20
     weighted avg       0.95      0.95      0.95        20



## Task 4 · Build channel features

In [23]:
I = np.eye(2, dtype=complex)
X = np.array([[0, 1], [1, 0]], dtype=complex)
Y = np.array([[0, -1j], [1j, 0]], dtype=complex)
Z = np.array([[1, 0], [0, -1]], dtype=complex)

def depolarizing_kraus(p):
    k0 = np.sqrt(1 - p) * I
    k1 = np.sqrt(p/3) * X
    k2 = np.sqrt(p/3) * Y
    k3 = np.sqrt(p/3) * Z
    return Kraus([k0, k1, k2, k3])

def amplitude_damping_kraus(gamma):
    k0 = np.array([[1, 0], [0, np.sqrt(1-gamma)]], dtype=complex)
    k1 = np.array([[0, np.sqrt(gamma)], [0, 0]], dtype=complex)
    return Kraus([k0, k1])

def channel_to_feature(channel):
    choi = Choi(channel).data
    feat = np.concatenate([choi.real.flatten(), choi.imag.flatten()])
    return feat


## Task 5 · Classify sample channels

In [24]:
channels = [
    ('depolarizing_p0.1', depolarizing_kraus(0.1)),
    ('depolarizing_p0.5', depolarizing_kraus(0.5)),
    ('amp_damp_0.1', amplitude_damping_kraus(0.1)),
    ('amp_damp_0.5', amplitude_damping_kraus(0.5)),
]

features = []
names = []
for name, ch in channels:
    f = channel_to_feature(ch)
    names.append(name)
    features.append(f)
X = np.vstack(features)

preds = model.predict(X)
df = pd.DataFrame({'channel': names, 'prediction': preds})
df


,channel,prediction
0,depolarizing_p0.1,amplitude_damping
1,depolarizing_p0.5,depolarizing
2,amp_damp_0.1,amplitude_damping
3,amp_damp_0.5,amplitude_damping
